In [1]:
from kss import KSS
from data_utils import *
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [2]:
MF = ['SEP','COH','WPC','CC','GP']

In [3]:
features = []
labels = []
names = []

datasets = [Ecoli(),Glass(),Haberman(),Ionosphere(),Iris(),Pima(),Sonar(),Thyroid(),Vehicle(),WDBC(),Wine()]
for data in datasets:
    f,l = data.get_data()
    features.append(f)
    labels.append(l)
    names.append(type(data).__name__)

In [4]:
results ={}
j=0

for i in range(len(names)):
    feature = features[i].to_numpy()
    label = labels[i].to_numpy()
    name = names[i]
    
    kf = KFold(n_splits=10, random_state=None, shuffle=True)
    k_fold_result = list(kf.split(feature,label))
    
    for mf_type in MF:
        f1_score = 0
        for train_index, test_index in k_fold_result:
            X_train, X_test = feature[train_index], feature[test_index]
            y_train, y_test = label[train_index], label[test_index]
            
            kss = KSS(7)
            if mf_type == 'GP':
                kss.fit(X_train,y_train,mass_type=mf_type,dataset_name=name.lower())
            else:
                kss.fit(X_train,y_train,mass_type=mf_type)
            y_pred = kss.predict(X_test)

            f1_score += classification_report(y_test, y_pred,output_dict=True)['macro avg']['f1-score']
        results[j] = [name,mf_type,f1_score/10]
        j+=1

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classificat

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classificat

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMe

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMe

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classificat

/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/site-packages/sklearn/metrics/_classificat

In [5]:
df = pd.DataFrame.from_dict(results,orient='index',columns=['Data sets','MF', "F1-Score"])

In [6]:
df

,Data sets,MF,F1-Score
0,Ecoli,SEP,0.762630
1,Ecoli,COH,0.701494
2,Ecoli,WPC,0.703202
3,Ecoli,CC,0.722495
4,Ecoli,GP,0.728980
5,Glass,SEP,0.889834
6,Glass,COH,0.905868
7,Glass,WPC,0.905868
8,Glass,CC,0.911011
9,Glass,GP,0.851310


In [7]:
df_SEP = df.loc[df['MF']=='SEP']
df_COH = df.loc[df['MF']=='COH']
df_WPC = df.loc[df['MF']=='WPC']
df_CC = df.loc[df['MF']=='CC']
df_GP = df.loc[df['MF']=='GP']

In [8]:
df_SEP = df_SEP.rename(columns={'F1-Score':'SEP-kSS'})
df_SEP = df_SEP.set_index('Data sets')
df_SEP = df_SEP.drop('MF', 1)


df_COH = df_COH.rename(columns={'F1-Score':'COH-kSS'})
df_COH = df_COH.set_index('Data sets')
df_COH = df_COH.drop('MF', 1)

df_WPC = df_WPC.rename(columns={'F1-Score':'WPC-kSS'})
df_WPC = df_WPC.set_index('Data sets')
df_WPC = df_WPC.drop('MF', 1)

df_CC = df_CC.rename(columns={'F1-Score':'CC-kSS'})
df_CC = df_CC.set_index('Data sets')
df_CC = df_CC.drop('MF', 1)

df_GP = df_GP.rename(columns={'F1-Score':'GP-kSS'})
df_GP = df_GP.set_index('Data sets')
df_GP = df_GP.drop('MF', 1)


In [9]:
df_organized = pd.concat([df_CC,df_WPC,df_COH,df_SEP,df_GP],axis=1)

df_organized.loc['MEAN'] = df_organized.mean()

df_organized = df_organized.round(4)

df_organized


,CC-kSS,WPC-kSS,COH-kSS,SEP-kSS,GP-kSS
Data sets,,,,,
Ecoli,0.7225,0.7032,0.7015,0.7626,0.7290
Glass,0.9110,0.9059,0.9059,0.8898,0.8513
Haberman,0.5899,0.6183,0.5827,0.5491,0.5807
Ionosphere,0.7783,0.9286,0.8282,0.6426,0.8199
Iris,0.9654,0.9749,0.9749,0.9749,0.9749
Pima,0.6834,0.6718,0.6839,0.6787,0.5599
Sonar,0.6959,0.7422,0.7285,0.7178,0.7178
Thyroid,0.8137,0.8568,0.8410,0.8137,0.8546
Vehicle,0.5956,0.6457,0.6390,0.6534,0.5861


In [10]:
print(df_organized.to_csv(index=True))

Data sets,CC-kSS,WPC-kSS,COH-kSS,SEP-kSS,GP-kSS
Ecoli,0.7225,0.7032,0.7015,0.7626,0.729
Glass,0.911,0.9059,0.9059,0.8898,0.8513
Haberman,0.5899,0.6183,0.5827,0.5491,0.5807
Ionosphere,0.7783,0.9286,0.8282,0.6426,0.8199
Iris,0.9654,0.9749,0.9749,0.9749,0.9749
Pima,0.6834,0.6718,0.6839,0.6787,0.5599
Sonar,0.6959,0.7422,0.7285,0.7178,0.7178
Thyroid,0.8137,0.8568,0.841,0.8137,0.8546
Vehicle,0.5956,0.6457,0.639,0.6534,0.5861
WDBC,0.6836,0.7289,0.7045,0.7102,0.3637
Wine,0.6417,0.6798,0.6703,0.6609,0.6609
MEAN,0.7346,0.7687,0.7509,0.7322,0.6999

